# Ejercicio EvCont4 (a) - Compresión LZZ7

* Óscar Sementé Solà
* Abdelkarim Azzouguagh Ouniri
* Rodrigo Cabezas Quirós

In [26]:
import re
msg = "11011100101001111010100010001"
[(m.start(0), m.end(0)) for m in re.finditer("1101", msg)]
#re.findall(r"1101", msg)

[(0, 4), (15, 19)]

In [61]:
import math, re

class LZZ7Compressor:
    
    def __init__(self, win, wsld):
        self.win = win
        self.wsld = wsld
        self.win_b = int(math.log2(win))
        self.wsld_b = int(math.log2(wsld))
    
    def __is_power_of_two(self, v):
        return math.log2(v).is_integer()
    
    def __ld_to_bin(self, l, d, useLD):
        b_win = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(self.win_b)]))
        b_wsld = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(self.wsld_b)]))
        if useLD:
            return (l, d)
        return b_win(l) + b_wsld(d)
    
    def __match_pattern(self, slide, inp, useLD):
        occ, seqs = [], [inp[:i] for i in range(len(inp), 0, -1)]
        while seqs:
            current = seqs.pop(0)
            occ = [(x.start(0), x.end(0)) for x in re.finditer(current, slide)]
            if occ:
                si, ei = occ[-1]
                return self.__ld_to_bin(len(slide[si:ei]), len(slide) - si, useLD), ei - si
        return False, 1
    
    def compress(self, m, useLD=False):
        ret = [m[:self.wsld]]
        
        if self.win <= self.wsld or len(m) >= self.win + self.wsld:
            """
            Que los bits de conf. sean potencia de 2 no tiene ningún sentido. El ejercicio evaluado en
            clase tenía por parametros 8 y 6 bits. Aplicando la norma de la potencia de 2, con 6 bits no
            funcionaria el algoritmo. Así que hemos aplicado esta política.
            or \ not self.__is_power_of_two(self.win) \ or not self.__is_power_of_two(self.wsld):
            """
            return -1
        # FALTA METER INSERCIÓN DE BIT PARA SECUENCIAS IGUALES ENTERAS
        offset = 0
        while offset + self.wsld + self.win <= len(m):
            slide = m[offset:offset + self.wsld] 
            inp = m[offset + self.wsld:offset + self.wsld + self.win]
            found, flen = self.__match_pattern(slide, inp, useLD)
            if found:
                ret.append(found)
            else:
                ret.append(self.__ld_to_bin(1, 1, useLD))
            offset -= -flen
            
        if flen < len(inp):
            ret.append(inp[flen:])
        
        if useLD:
            return ret
        return "".join(ret)
    

In [62]:
msg = "11011100101001111010100010001"
comp = LZZ7Compressor(6, 8)
print(comp.compress(msg, useLD=True))
print(comp.compress(msg))

-1
-1


#### [Apartado 1] Comprobad que el programa comprime y descomprime correctamente una cadena de 25 bits aleatorios con Mdes = 8 y Ment = 4. Ayuda: Podéis generar bits aleatorios con Math.round(Math.random()).

#### [Apartado 2] Utilizad el programa anterior para determinar si es posible, ajustando los valores de Mdes y Ment, conseguir comprimir datos aleatorios mediante LZ77 (es decir, que la cadena de datos originales sea más larga que la cadena comprimida). ¿Por qué? ¿Cuál es la máxima compresión que lográis? ¿Con qué valores? (Ayuda: utilizad una cadena de datos de entrada de, por lo menos, 10000 bits aleatorios. Ajustad Mdes y Ment entre 2 y 2048).